In [6]:
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display

In [2]:
HOME = os.environ.get('HOME')


import analysis as A
import common as C

%load_ext autoreload
%autoreload 2

/home/asem/GP/ICENODE/icenode-env/lib/python3.8/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '


In [3]:
mimic_dir = {
    'M3': f'{HOME}/GP/ehr-data/mimic3-transforms',
    'M4': f'{HOME}/GP/ehr-data/mimic4-transforms'
}

train_dir = {
    'GM3': f'{HOME}/GP/ehr-data/icenode-m3-exp/train_config_v0.2.25_G_M3',
    'M3': f'{HOME}/GP/ehr-data/icenode-m3-exp/train_config_v0.2.25_M3',
    'M4': f'{HOME}/GP/ehr-data/icenode-m4-exp/train_config_v0.2.25_M4',
    'GM4': f'{HOME}/GP/ehr-data/icenode-m4-exp/train_config_v0.2.25_G_M4'
}

model_dir = {
    'ICE-NODE': 'icenode_2lr',
    'ICE-NODE_UNIFORM': 'icenode_uniform2lr',
    'GRU': 'gru',
    'RETAIN': 'retain'
}


clfs = list(model_dir.keys())


In [ ]:
m4_interface = C.create_patient_interface(mimic_dir['M4'], 'M4')
m3_interface = C.create_patient_interface(mimic_dir['M3'], 'M3')

m4_train_ids, m4_valid_ids, m4_test_ids = m4_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)
m3_train_ids, m3_valid_ids, m3_test_ids = m3_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)


In [29]:

def get_trained_models(clfs, train_dir, model_dir, data_tag):
    clfs_params_dir = train_dir[data_tag]
    for clf in clfs:
        clf_dir = model_dir[clf]
        clf_cls = C.model_cls[clf]
        param_files = sorted(
            glob.glob(f'{clfs_params_dir}/{clf_dir}/*params.pickle', recursive=False))
        
        for param_file in param_files:
            params = C.load_params(param_file)
            if 'diag_emb' in params:
                print('x')
                params['dx_emb'] = params['diag_emb']
                del params['diag_emb']
                C.write_params(params, param_file)
#             print(params.keys())

        
#         params[clf] = C.load_params(params_file)
#         config[clf] = C.load_config(f'{clfs_params_dir}/{clf_dir}/config.json')


In [30]:
get_trained_models(clfs, train_dir, model_dir, 'M3')

In [31]:
get_trained_models(clfs, train_dir, model_dir, 'M4')

In [32]:
get_trained_models(clfs, train_dir, model_dir, 'GM3')

In [33]:
get_trained_models(clfs, train_dir, model_dir, 'GM4')